---
title: |
  Gestionando tu proyecto con
  ![](images/mlflow.png){width="60%"}  
	
theme: Montpellier
author: "Daniel Molina Cabrera"
date: 19 December 2024 
date-format: long
toc: false
toc-title: Tabla de Contenidos
toc-depth: 1
slide-number: true
execute:
  echo: true
  cache: true
output:
  beamer_presentation:
    slide_level: 1
format:
  html:
    code-fold: false
    code-summary: "Show Code"
    fig-width: 5
    fig-height: 3
    fig-align: center
    fig-justify: center
  beamer:
    fig-width: 4
    fig-height: 2
  revealjs:
    theme: white
    fig-align: center
    fig-height: 5
    smaller: true
    logo: images/license.png
    slide-number: true
---

## Material de las transparencias

::: {.columns}

:::: {.column width="50%"}

::::: {style="text-align:center;"}
![Github](images/qr_github.png){width="100%" #github}
:::::

::::

:::: {.column width="50%"}

::::: {style="text-align:center;"}
![Slides](images/qr_slides.png){width="100%" #slides}
:::::

::::
:::

## Sobre mí {.smaller}

::: {.columns}
![](images/logos.png){fig-align="top" width="100%"}

:::: {.column width="40%"}
![](images/yo_serio.png){width="80%"}
::::

:::: {.column width="60%"}

- Profesor Titular de la Universidad de Granada.
- Investigador en Inteligencia Artificial.
- Enseñando ML varios años usando Python y R.
- Usando técnicas de ML en proyectos con empresas.

![](images/DaSCI_logo_1.png){width="40%"}
::::

:::

# ¿Qué hace?

## ¿Cuál es el problema que intenta resolver?

. . .

::: {.columns}

:::: {.column width="60%"}

![](images/coupled_confused.jpg)
::::

:::: {.column width="40%"}

![](images/preguntas.jpg){width="60%"}

- Múltiples _notebooks_ y modelos.
- Cuál obtiene cada resultado.
- Reproducibilidad.
::::

:::

## ¿Qué se necesitaría? {.bigger}

- Asociar los resultados a cada modelo/parámetros.

- Registrar los modelos ya preparados.

- Registrar los datos usados.

- Poder ejecutar todo con distintos parámetros y/o datos.

. . .

![](images/mlflow_options.png){width="80%"}

## MLFlow integrado en el flujo de trabajo

![](images/mlops-mlflow.png)

## MLFlow conoce las librerías más usadas de ML

![](images/integracion.png)

# Verlo en acción

## Primero hay que instalarlo

Lo podemos instalar directamente con pip.

In [ ]:
#| output: false
! pip install mlflow

Podemos indicar dónde guardar los datos:

```bash
$ export MLFLOW_TRACKING_URI=sqlite:///datos.db
```

y luego ejecutarlo con:

```bash
$ mlflow server  mlflow server --backend-store-uri sqlite:///datos.db
...[INFO] Starting gunicorn 23.0.0
...[INFO] Listening at: http://127.0.0.1:5000 (77581)
...[INFO] Using worker: sync
...[INFO] Booting worker with pid: 77582
...[INFO] Booting worker with pid: 77583
...[INFO] Booting worker with pid: 77584
...[INFO] Booting worker with pid: 77585
```

## Acceso a MLFlow

Podemos acceder, pero todavía no hay nada interesante.

[![](images/mlflow1.png)](http://localhost:5000)

## Primeros ejemplos

Partimos de un código sencillo:

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


def main():
    iris = load_iris()

    # Testing different maximum-depth values
    for max_depth in range(1, 6):
        # Create a decision tree classifier
        clf = DecisionTreeClassifier(max_depth=max_depth)

        # Perform cross-validation with 5 folds
        scores = cross_val_score(clf, iris.data, iris.target, cv=5).mean()

        # Print the cross-validation scores
        print(f"Cross-validation scores for max_depth={max_depth}: {scores:.5f}")


main()

## Registrando el experimento

Vamos a incorporar el uso de MLFlow inicialmente. Eso implica:

1. Indicar dónde se guardarán los datos.

```python
mlflow.set_tracking_uri("http://127.0.0.1:5000")
```

2. Crear un experimento asignándole un nombre:
```python
mlflow.set_experiment("Iris - max_depth study")
```

3. Delimitar el código asociado a cada ejecución

```python
with mlflow.start_run():
    ...
```

Para evitar nombres de ejecución aleatorios, le pondremos nombre a cada.

4. Guardar los parámetros:

```python
mlflow.log_param("max_depth", max_depth)
```

5. Guardar los resultados:

```python
mlflow.log_metric("accuracy", scores)
```

## Probando el ejemplo completo

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import mlflow


def main():
    iris = load_iris()
    mlflow.set_tracking_uri("http://127.0.0.1:5000")

    mlflow.set_experiment("Iris - max_depth study")

    for max_depth in range(1, 5):
        with mlflow.start_run():
            mlflow.log_param("max_depth", max_depth)

            clf = DecisionTreeClassifier(max_depth=max_depth)
            scores = cross_val_score(clf, iris.data, iris.target, cv=5).mean()
            print(f"Cross-validation scores for max_depth={max_depth}: {scores:.5f}")
            # Registro la métrica
            mlflow.log_metric("accuracy", scores)


main()

## Volvemos a la página anterior y cambiamos las columnas

[![](images/mlflow2.png)](http://localhost:5000)

[![](images/mlflow4.png)](http://localhost:5000)

## Añadiendo más métricas

Permite gestionar muchas métricas a la vez:

## Mejoras:

1. Guardar la información del modelo:

```python
mlflow.sklearn.log_model(
                sk_model=clf,
                artifact_path="decision-tree",
                registered_model_name="dc-reg-model",
            )
```

2. Usemos múltiples métricas.

```python
            scores = cross_validate(
                clf,
                cancer.data,
                cancer.target,
                cv=5,
                scoring=["accuracy", "f1", "recall", "precision", "roc_auc"],
            )

            for score in scores:
                # Registro la métrica
                mlflow.log_metric(score, scores[score].mean())
```

## Todo junto

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
import mlflow

def main():
    cancer = load_breast_cancer()
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    mlflow.set_experiment("Cancer - max_depth study")

    # Testing different maximum-depth values
    for max_depth in range(1, 5):
        with mlflow.start_run(run_name=f"max_depth-{max_depth}"):
            mlflow.log_param("max_depth", max_depth)
            clf = DecisionTreeClassifier(max_depth=max_depth)

            scores = cross_validate(
                clf,
                cancer.data,
                cancer.target,
                cv=5,
                scoring=["accuracy", "f1", "recall", "precision", "roc_auc"],
            )

            for score in scores:
                print(score)
                print(scores[score].mean())
                # Registro la métrica
                mlflow.log_metric(score, scores[score].mean())

            # Log the sklearn model and register as version 1
            mlflow.sklearn.log_model(
                sk_model=clf,
                artifact_path="decision-tree",
                registered_model_name="dc-reg-model",
            )

main()

## Vamos a ver qué tal va

[![](images/mlflow6.png)](http://localhost:5000)

# Es muy pesado, ¿se puede simplificar?

## Auto-logging

- MLFlow conoce la mayoría de librerías: Scikit-learn, Keras, PyTorch, XGBoost, ...

- Permite registrar de forma automática.

```python
    # Creo el experimento si no existe
    mlflow.set_experiment("Cancer - max_depth study auto")
    # Activo el autolog
    mlflow.autolog()
```

- Registrará cada modelo, con los distintos parámetros.

## Ejemplo

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
import mlflow

def main():
    cancer = load_breast_cancer()
    # Añadimos dónde se guardarán los datos
    mlflow.set_tracking_uri("http://127.0.0.1:5000")

    # Creo el experimento si no existe
    mlflow.set_experiment("Cancer - max_depth study auto")
    mlflow.autolog()

    # Testing different maximum-depth values
    for max_depth in range(1, 5):
        # Con with no es necesario iniciar y cerrar, es más cómodo
        with mlflow.start_run(run_name=f"max_depth-{max_depth}"):
            # Create a decision tree classifier
            clf = DecisionTreeClassifier(max_depth=max_depth)

            # Perform cross-validation with 5 folds
            scores = cross_validate(
                clf,
                cancer.data,
                cancer.target,
                cv=5,
                scoring=["accuracy", "f1", "recall", "precision", "roc_auc"],
            )

            for score in scores:
                print(score)
                print(scores[score].mean())


main()


## Resultado

[![](images/mlflowa.png)](http://localhost:5000){width="120%"}

# Comparando

## Podemos comparar las distintas salidas

[![](images/mlflow7.png)](http://localhost:5000)

# Reusando el modelo

## Reusando el modelo

El modelo no solo se puede registrar de cara a comparar diferencias entre ellos.

Si se registra se puede recuperar simplemente con:

```python
    model = mlflow.sklearn.load_model(f"models:/<nombre_modelo>/<versión>")
```

## Ejemplo de carga de modelo

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
import mlflow


def main():
    cancer = load_breast_cancer()
    # Añadimos dónde se guardarán los datos
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    model = mlflow.sklearn.load_model(f"models:/dc-reg-model/4")

    # Testing different maximum-depth values
    for max_depth in range(1, 5):
        # Perform cross-validation with 5 folds
        scores = cross_validate(
            model,
            cancer.data,
            cancer.target,
            cv=5,
            scoring=["accuracy", "f1", "recall", "precision", "roc_auc"],
        )

        for score in scores:
            print(score)
            print(scores[score].mean())


main()

## Hay mucho más

- Registrar los _datasets_ (aunque no es lo mejor que ofrece).

- Uso de proyectos (dependencias, ejecución de proyecto enteramente en git).

- Desplegar en sistemas de despliegue (docker, kubernete, AWS, ...).

# Otras opciones

- [Weights and Bias](https://wandb.ai/site)

- [KubeFlow](https://www.kubeflow.org/)

- [neptune.ai](neptune.ai)

# Muchas gracias

::: {.columns  style='display: flex !important; height: 90%;'} 


:::: {.column width="80%"   style='display: flex; justify-content: center; align-items: center;'}
![](images/minions_gracias.gif){ width="100%" fig-align=center}
::::

:::: {.column width="20%"   style='display: flex; justify-content: center; align-items: center;'}
![](images/preguntas.png){ width="60%" fig-align=center}
::::

:::

::: {.columns  style='display: flex !important; height: 40%;'} 

:::: {.column width="10%"   style='display: flex; justify-content: right; align-items: right;'}
![](images/yo.png){width="30%" fig-align="right" }
::::
:::: {.column width="80%" }
[dmolinac@ugr.es](mailto:dmolinac@ugr.es)

[http://www.danimolina.net](http://www.danimolina.net)
::::